## Create a Program:

The program should simulate 1000 individual battle rounds in the board game Risk, specifically with 3 attackers vs. 2 defenders.

### Battle round rules: 
Each battle round involves one roll of dice for both attacker and defender. 
**The highest dice roll from each side is compared:**
If the attacker’s roll is equal to or lower than the defender’s, *the attacker loses one troop*.
If the attacker’s roll is higher, *the defender loses one troop*.
**Compare Second-Highest Dice Rolls**: The same rule. 

### Plot Results: 
The program should produce a plot summarizing the results of the battle rounds. The specifics of what to plot are left to your discretion.

Extra Challenge: Create a more complex simulation that runs until one side's army is eliminated. This simulation should handle armies of any size and plot the outcome.

This program should model battle outcomes, track results, and create visualizations of the simulated rounds.

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [85]:
#Ref.:https://www.w3schools.com/python/numpy/numpy_random.asp
# https://numpy.org/doc/2.0/reference/random/generated/numpy.random.seed.html
 

# I need to create two generators, one for attacker and another for defender. 
#np.random.randint can generate 2D arrays
#1 and 7 is min and max; size is an array size.

#attacker's results 
np.random.seed(4)
dice_roll_attacker= np.random.randint(1,7, size=(1000,3))
print(f'Attacker’s turns: first five rows:\n{dice_roll_attacker[0:5,:]}')

#defender's results 
np.random.seed(4)
dice_roll_defender= np.random.randint(1,7, size=(1000,2))
print(f'Defender’s turns: first five rows:\n{dice_roll_defender[0:5,:]}')


Attacker’s turns: first five rows:
[[3 6 2]
 [1 1 3]
 [2 3 5]
 [6 2 1]
 [5 3 5]]
Defender’s turns: first five rows:
[[3 6]
 [2 1]
 [1 3]
 [2 3]
 [5 6]]


In [86]:
# Ref.:https://www.w3schools.com/python/numpy/numpy_array_sort.asp

# Attacker has 3 dice and I need to take for comparison 2 biggest numbers.
# I will sort the array.
# axis=1 for sorting by raws
# [:, ::-1] - to invert the order
# expression ::-1 refers to the elements within each row.
# This means that we want to select all elements, but in reverse order.

dice_roll_attacker = np.sort(dice_roll_attacker, axis=1 )[:, ::-1]
print(f'Sorted attacker\'s turns. First five rows:\n{dice_roll_attacker[0:5,:]}')

Sorted attacker's turns. First five rows:
[[6 3 2]
 [3 1 1]
 [5 3 2]
 [6 2 1]
 [5 5 3]]


In [87]:
#Now, when it sorted, I can take the rows with bigger numbers.
two_largest_attack= dice_roll_attacker[:,0:2]
print(f'Sorted and filtered atacker\'s turns. first five rows:\n{two_largest_attack[0:5,:]}')

Sorted and filtered atacker's turns. first five rows:
[[6 3]
 [3 1]
 [5 3]
 [6 2]
 [5 5]]


In [88]:
#Sorting defender's turns.
dice_roll_defender = np.sort(dice_roll_defender, axis=1 )[:,::-1]
print(f'Sorted defender\'s turns. First five rows:\n{dice_roll_defender[0:5,:]}')

Sorted defender's turns. First five rows:
[[6 3]
 [2 1]
 [3 1]
 [3 2]
 [6 5]]
